#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 2, 9)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,33,S,IMH,"3,000",20.70,2,15pct,mai


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
2,B,BGRIM,-2,52L,SET50,"3,000",33.00,33.50,-1.25,-3.60%,2,Buy,"99,000.00"
19,B,TQM,-2,3L,SET100,"4,500",45.75,46.25,-0.25,-0.54%,2,Buy,"205,875.00"
22,S,CPNCG,1,DOS,SET,"10,000",12.80,12.70,0.10,+0.79%,2,Sell,"128,000.00"
24,S,IVL,4,20pct,SET50,"1,200",50.50,49.50,0.00,0.00%,2,Sell,"60,600.00"
25,S,KCE,48,CP1S,SET50,"2,000",85.25,73.25,1.00,+1.38%,2,Sell,"170,500.00"
27,S,TOP,1,10pct,SET50,"2,000",54.00,53.75,-0.25,-0.46%,2,Sell,"108,000.00"
30,S,TU,4,5pct,SET50,"6,000",21.80,21.40,0.20,+0.94%,2,Sell,"130,800.00"


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,BGRIM,-2,52L,SET50,"3,000",33.00,33.50,-1.25,-3.60%,2,Buy,"99,000.00"
B,BGC,-6,DOS,SET,"10,000",10.00,10.60,0.10,+0.95%,1,Buy,"100,000.00"
B,ASP,-8,1L,SET,"30,000",3.66,3.82,-0.04,-1.04%,1,Buy,"109,800.00"
B,EPG,-9,RD05pct,SET100,"6,000",9.90,10.70,0.20,+1.90%,1,Buy,"59,400.00"
B,GLOBAL,-11,RD05pct,SET50,"7,500",18.20,19.30,0.20,+1.05%,0,Buy,"136,500.00"


### End of Tables in the process

### Print to verify before upload file

In [30]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [31]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [10]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGRIM,"3,000",33.00,"99,000.00",52L,SET50,2
1,B,PTTEP,"1,200",128.50,"154,200.00",DOS,SET50,2
2,B,TQM,"4,500",45.75,"205,875.00",3L,SET100,2
3,S,CPNCG,"10,000",12.80,"128,000.00",DOS,SET,2
4,S,IMH,"3,000",20.70,"62,100.00",15pct,mai,2
5,S,IVL,"1,200",50.50,"60,600.00",20pct,SET50,2
6,S,TOP,"2,000",54.00,"108,000.00",10pct,SET50,2
7,S,TU,"6,000",21.80,"130,800.00",5pct,SET50,2


In [11]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    459075.0
S    489500.0
Name: amount, dtype: float64

In [12]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGRIM,"3,000",33.00,"99,000.00",52L,SET50,2
1,B,PTTEP,"1,200",128.50,"154,200.00",DOS,SET50,2
2,B,TQM,"4,500",45.75,"205,875.00",3L,SET100,2


In [13]:
cash = 504_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(504000, 459075.0, 44925.0)

In [14]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
3,S,CPNCG,"10,000",12.80,"128,000.00",DOS,SET,2
4,S,IMH,"3,000",20.70,"62,100.00",15pct,mai,2
5,S,IVL,"1,200",50.50,"60,600.00",20pct,SET50,2
6,S,TOP,"2,000",54.00,"108,000.00",10pct,SET50,2
7,S,TU,"6,000",21.80,"130,800.00",5pct,SET50,2


In [15]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(504000, 489500.0, 993500.0)

### After call ord-log (must call everytime that orders change)

In [16]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [17]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [20]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
1,B,BGC,-5,DOS,SET,"10,000",10.00,10.50,-0.10,-0.94%,1,Buy,"100,000.00"
2,B,BGRIM,-4,52L,SET50,"3,000",33.00,34.00,0.50,+1.49%,2,Buy,"99,000.00"
9,B,PTTEP,-5,DOS,SET50,"1,200",128.50,131.00,-3.00,-2.24%,2,Buy,"154,200.00"
15,B,SINGER,-4,15%,SET100,"4,500",45.00,46.00,-0.25,-0.54%,1,Buy,"202,500.00"


In [22]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [23]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
22,S,CPNCG,1,DOS,SET,"10,000",12.80,12.70,0.00,0.00%,2,Sell,"128,000.00"
25,S,IVL,2,20pct,SET50,"1,200",50.50,50.00,0.50,+1.01%,2,Sell,"60,600.00"
28,S,TOP,3,10pct,SET50,"2,000",54.00,53.25,-0.50,-0.93%,2,Sell,"108,000.00"


In [24]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
1,B,BGC,-5,DOS,SET,10000,10.0,10.50,-0.10,-0.94%,1,Buy,100000.0
2,B,BGRIM,-4,52L,SET50,3000,33.0,34.00,0.50,+1.49%,2,Buy,99000.0
9,B,PTTEP,-5,DOS,SET50,1200,128.5,131.00,-3.00,-2.24%,2,Buy,154200.0
15,B,SINGER,-4,15%,SET100,4500,45.0,46.00,-0.25,-0.54%,1,Buy,202500.0
22,S,CPNCG,1,DOS,SET,10000,12.8,12.70,0.00,0.00%,2,Sell,128000.0
25,S,IVL,2,20pct,SET50,1200,50.5,50.00,0.50,+1.01%,2,Sell,60600.0
28,S,TOP,3,10pct,SET50,2000,54.0,53.25,-0.50,-0.93%,2,Sell,108000.0


In [25]:
df[mb | ms].shape[0]

7

In [26]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
15,SINGER,Buy,"202,500.00"
9,PTTEP,Buy,"154,200.00"
22,CPNCG,Sell,"128,000.00"
28,TOP,Sell,"108,000.00"
1,BGC,Buy,"100,000.00"


### Select source of orders between these two

In [27]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent
1,Buy,BGC,"10,000",10.00,1,-5,10.50,-0.10,-0.94%
2,Buy,BGRIM,"3,000",33.00,2,-4,34.00,0.50,+1.49%
9,Buy,PTTEP,"1,200",128.50,2,-5,131.00,-3.00,-2.24%
15,Buy,SINGER,"4,500",45.00,1,-4,46.00,-0.25,-0.54%
22,Sell,CPNCG,"10,000",12.80,2,1,12.70,0.00,0.00%
25,Sell,IVL,"1,200",50.50,2,2,50.00,0.50,+1.01%
28,Sell,TOP,"2,000",54.00,2,3,53.25,-0.50,-0.93%


In [28]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent
2,Buy,BGRIM,"3,000",33.00,2,-4,34.00,0.50,+1.49%
9,Buy,PTTEP,"1,200",128.50,2,-5,131.00,-3.00,-2.24%
19,Buy,TQM,"4,500",45.75,2,-6,47.25,1.00,+2.16%
22,Sell,CPNCG,"10,000",12.80,2,1,12.70,0.00,0.00%
23,Sell,IMH,"3,000",20.70,2,9,19.80,0.90,+4.76%
25,Sell,IVL,"1,200",50.50,2,2,50.00,0.50,+1.01%
28,Sell,TOP,"2,000",54.00,2,3,53.25,-0.50,-0.93%
31,Sell,TU,"6,000",21.80,2,6,21.20,-0.20,-0.93%


In [29]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process